# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 6 2022 9:17AM,241173,10,CLI00320.1,"CLINUVEL, Inc.",Released
1,Jul 6 2022 9:13AM,241171,10,Iron - 1097,Prodigy Innovation LLC,Released
2,Jul 6 2022 9:12AM,241170,10,CTF0011301,"Citieffe, Inc.",Released
3,Jul 6 2022 9:08AM,241164,10,PRF-32751,Bio-PRF,Released
4,Jul 6 2022 9:08AM,241164,10,PRF-32765,Bio-PRF,Released
5,Jul 6 2022 9:08AM,241164,10,PRF-32768,Bio-PRF,Released
6,Jul 6 2022 9:08AM,241164,10,PRF-32773,Bio-PRF,Released
7,Jul 6 2022 9:08AM,241164,10,PRF-32778,Bio-PRF,Released
8,Jul 6 2022 9:08AM,241164,10,PRF-32782,Bio-PRF,Released
9,Jul 6 2022 9:08AM,241164,10,PRF-32786,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
70,241164,Released,8
71,241165,Released,1
72,241170,Released,1
73,241171,Released,1
74,241173,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
241164,NaN,NaN,8.0
241165,NaN,NaN,1.0
241170,NaN,NaN,1.0
241171,NaN,NaN,1.0
241173,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241045,0.0,2.0,0.0
241049,0.0,0.0,55.0
241050,0.0,0.0,55.0
241055,0.0,0.0,55.0
241056,0.0,0.0,49.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
241045,0,2,0
241049,0,0,55
241050,0,0,55
241055,0,0,55
241056,0,0,49


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241045,0,2,0
1,241049,0,0,55
2,241050,0,0,55
3,241055,0,0,55
4,241056,0,0,49


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,241045,,2,
1,241049,,,55
2,241050,,,55
3,241055,,,55
4,241056,,,49


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc."
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc."
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF
11,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation
69,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc
70,Jul 6 2022 8:40AM,241161,10,ISDIN Corporation
114,Jul 6 2022 8:30AM,241145,10,ISDIN Corporation
118,Jul 6 2022 8:30AM,241095,10,Eywa Pharma Inc.
126,Jul 6 2022 8:30AM,241147,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc.",,,1
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,,,1
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",,,1
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF,,,8
4,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,,,58
5,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc,,,1
6,Jul 6 2022 8:40AM,241161,10,ISDIN Corporation,,,44
7,Jul 6 2022 8:30AM,241145,10,ISDIN Corporation,,,4
8,Jul 6 2022 8:30AM,241095,10,Eywa Pharma Inc.,,,8
9,Jul 6 2022 8:30AM,241147,10,"Snap Medical Industries, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc.",1,,
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,1,,
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",1,,
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF,8,,
4,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,58,,
5,Jul 6 2022 8:48AM,241165,18,Eye Pharma Inc,1,,
6,Jul 6 2022 8:40AM,241161,10,ISDIN Corporation,44,,
7,Jul 6 2022 8:30AM,241145,10,ISDIN Corporation,4,,
8,Jul 6 2022 8:30AM,241095,10,Eywa Pharma Inc.,8,,
9,Jul 6 2022 8:30AM,241147,10,"Snap Medical Industries, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc.",1,,
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,1,,
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",1,,
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF,8,,
4,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,58,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc.",1.0,NaN,NaN
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,1.0,NaN,NaN
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",1.0,NaN,NaN
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF,8.0,NaN,NaN
4,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,58.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 6 2022 9:17AM,241173,10,"CLINUVEL, Inc.",1.0,0.0,0.0
1,Jul 6 2022 9:13AM,241171,10,Prodigy Innovation LLC,1.0,0.0,0.0
2,Jul 6 2022 9:12AM,241170,10,"Citieffe, Inc.",1.0,0.0,0.0
3,Jul 6 2022 9:08AM,241164,10,Bio-PRF,8.0,0.0,0.0
4,Jul 6 2022 9:03AM,241163,10,ISDIN Corporation,58.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,10126801,1046.0,1.0,3.0
12,1205452,18.0,2.0,0.0
16,1929009,13.0,4.0,0.0
18,482320,2.0,0.0,0.0
19,723389,86.0,16.0,2.0
20,1205453,97.0,15.0,28.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,10126801,1046.0,1.0,3.0
1,12,1205452,18.0,2.0,0.0
2,16,1929009,13.0,4.0,0.0
3,18,482320,2.0,0.0,0.0
4,19,723389,86.0,16.0,2.0
5,20,1205453,97.0,15.0,28.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,1046.0,1.0,3.0
1,12,18.0,2.0,0.0
2,16,13.0,4.0,0.0
3,18,2.0,0.0,0.0
4,19,86.0,16.0,2.0
5,20,97.0,15.0,28.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1046.0
1,12,Released,18.0
2,16,Released,13.0
3,18,Released,2.0
4,19,Released,86.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Completed,3.0,0.0,0.0,0.0,2.0,28.0
Executing,1.0,2.0,4.0,0.0,16.0,15.0
Released,1046.0,18.0,13.0,2.0,86.0,97.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0
1,Executing,1.0,2.0,4.0,0.0,16.0,15.0
2,Released,1046.0,18.0,13.0,2.0,86.0,97.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Completed,3.0,0.0,0.0,0.0,2.0,28.0
1,Executing,1.0,2.0,4.0,0.0,16.0,15.0
2,Released,1046.0,18.0,13.0,2.0,86.0,97.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()